In [1]:
import torch
print(torch.__version__)

c:\Users\q1062\miniconda3\envs\SpeechVision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.0


In [2]:
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests



In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')    
print(device)

cuda:0


In [4]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [6]:
import pickle
# with open('embeddings.pkl', 'wb') as f:
#     pickle.dump(dict_visual_embeddings, f)


with open('embeddings.pkl', 'rb') as f:
     embeddings = pickle.load(f)

In [7]:
stackembeddings = []
for i in range(10):
    stackembeddings.append(torch.cat(embeddings[i]))

In [8]:
stackembeddings = torch.cat(stackembeddings)

In [12]:
transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

data_viz = torchvision.datasets.ImageFolder(
    "raw-img",
    transform=transform
)


In [31]:
def get_image(text):
    text=clip.tokenize([text]).to(device)
    text_features = model.encode_text(text)
    scores = torch.einsum('ab, cb -> ac', stackembeddings, text_features)
    idx = torch.argmax(scores)
    plt.imshow(np.transpose(data_viz[idx][0], (1,2,0)))
    plt.axis("off")
    plt.savefig('generateio')

In [32]:
get_image('a dog')

TypeError: Cannot handle this data type: (1, 1, 3), <f4

In [30]:
np.transpose(data_viz[0][0], (1,2,0)).numpy()

array([[[0.5921569 , 0.57254905, 0.56078434],
        [0.5882353 , 0.5686275 , 0.5529412 ],
        [0.58431375, 0.56078434, 0.54509807],
        ...,
        [0.92941177, 0.8862745 , 0.8627451 ],
        [0.92941177, 0.8862745 , 0.8627451 ],
        [0.92941177, 0.8862745 , 0.8627451 ]],

       [[0.59607846, 0.5764706 , 0.5647059 ],
        [0.5921569 , 0.57254905, 0.56078434],
        [0.58431375, 0.5647059 , 0.5529412 ],
        ...,
        [0.92941177, 0.8862745 , 0.8627451 ],
        [0.92941177, 0.8862745 , 0.8627451 ],
        [0.92941177, 0.8862745 , 0.8627451 ]],

       [[0.59607846, 0.5803922 , 0.5686275 ],
        [0.5921569 , 0.5764706 , 0.5647059 ],
        [0.58431375, 0.5686275 , 0.5568628 ],
        ...,
        [0.9254902 , 0.88235295, 0.85882354],
        [0.9254902 , 0.88235295, 0.85882354],
        [0.9254902 , 0.88235295, 0.85882354]],

       ...,

       [[0.50980395, 0.50980395, 0.5019608 ],
        [0.5058824 , 0.5058824 , 0.49803922],
        [0.5019608 , 0